# Knowledge Representation on the Web -- RDF tutorial

In this tutorial we'll learn the basics of interacting with RDF graphs with Python. We'll be using rdflib for this, a widely used Ptyhon library for RDF (all documentation can be found [here](https://rdflib.readthedocs.io/en/stable/index.html))

## Imports
These are the main classes and types we'll be using from rdflib

In [41]:
import sys
!{sys.executable} -m pip install rdflib

from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef, RDFS
from rdflib.namespace import DC, FOAF, XSD

import pprint


fish: Unknown command: truetrue
~/.config/fish/config.fish (line 1): 
truetrue
^
from sourcing file ~/.config/fish/config.fish
	called during startup


## Loading data remotely and from files

rdflib accepts importing RDF data from a variety of sources, either locally from a file (including an extensive support of serializations), or remotely via a URI (this is a great way of checking practically if URIs return RDF according to the 3rd Linked Data principle).

A Graph object is always required to load triples.
**Note**: to load quads, and hence supporting named graphs, you'll need to use an instance of ConjunctiveGraph instead

**Exercise 1** 

For each step, use a different cell: 
1. create two graphs using rdflib:
    - and load one with triples from the site https://csarven.ca/ and/or http://www.w3.org/People/Berners-Lee/card 
    - load one with triples from ./data/ingredients.rdf. 

In [42]:
g = Graph()
g.parse("https://www.w3.org/People/Berners-Lee/card")

g2 = Graph()
g2.parse("./data/ingredients.rdf")

<Graph identifier=Ned64dfdf4cef4541a6088a33e6c4ff08 (<class 'rdflib.graph.Graph'>)>

## Serialising and saving RDF graphs

There are different formats for storing RDF triples. Semantically, these mean the same, they differ only in their syntax. 


Use the function Graph.serialize(format). 

**Exercise 2**

1. serialise one of the graphs to the .ttl, .xml and .nt format, and print the first n lines to compare the syntax
1. save your graph in the turtle format to the ./data/ folder

In [43]:
print(g2.serialize(format="xml"))
print(g2.serialize(format="nt"))
print(g2.serialize(format="ttl"))


g2.serialize(destination="./data/ingredients.ttl")

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:dcterms="http://purl.org/dc/terms/"
   xmlns:owl="http://www.w3.org/2002/07/owl#"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
   xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
   xmlns:skos="http://www.w3.org/2004/02/skos/core#"
   xmlns:sm="https://www.omg.org/techprocess/ab/SpecificationMetadata/"
   xmlns:wtm="http://purl.org/heals/food/"
>
  <rdf:Description rdf:about="http://purl.org/heals/ingredient/User15">
    <rdf:type rdf:resource="http://www.w3.org/2002/07/owl#NamedIndividual"/>
    <rdf:type rdf:resource="http://purl.org/heals/food/User"/>
    <wtm:dislikes rdf:resource="http://purl.org/heals/ingredient/Almond"/>
    <wtm:dislikes rdf:resource="http://purl.org/heals/ingredient/Celery"/>
    <wtm:forbids rdf:resource="http://purl.org/heals/ingredient/Onion"/>
    <wtm:isAllergicTo rdf:resource="http://purl.org/heals/ingredient/Pecan"/>
    <dcterms:source>Techopedia, "User Identification (User ID)," 

<Graph identifier=Ned64dfdf4cef4541a6088a33e6c4ff08 (<class 'rdflib.graph.Graph'>)>

##  Merging graphs

Merging graphs can be done via sequential parsings or by the overloaded operator +

**Note:** Set-theoretic graph semantics apply

The Food knowledge graph FoodKG contains a graph of statements about ingredients, as well as a graph with statements about recipes. 

**Exercise 3**: 

1. load ./data/ingredients.rdf and ./data/ghostbusters.ttl into a single graph, either by sequential parsing or using the operator +.

2. count the number of statements in each graph, and the intersection of the two graphs. 

3. check whether the combined graph is connected (using graph.connected()) 

4. load ./data/ingredients.rdf and ./data/recipes.rdf into a single graph, either by sequential parsing or using the operator +. 

5. count the number of statements in each graph, and the intersection of the two graphs. 

6. check whether the combined graph is connected (using graph.connected()). Explain the result with respect to point 3! 

In [44]:
# g3 = Graph()
# g3.parse("./data/ingredients.rdf")
# print("First:", len(g3))
# g3.parse("./data/ghostbusters.ttl")
# print("Second: ", len(g3))
# res3 = g3.connected()
# print(res3)

g1 = Graph().parse("./data/ingredients.rdf")
g2 = Graph().parse("./data/ghostbusters.ttl")

g3 = g1 & g2
print("Done merging")
print(g3.connected())


g4 = Graph()
g4.parse("./data/ingredients.rdf")
g4.parse("./data/recipes.rdf")
res = g4.connected()
print(res)

Done merging
False
False


## Namespaces 

Remind yourself what namespaces are. 

In RDFLib, the namespace module defines many common namespaces such as RDF, RDFS, OWL, FOAF, SKOS, etc., but you can also easily add URIs within a different namespace:


In [45]:
TEACH = Namespace("http://linkedscience.org/teach/ns#")
TEACH.Teacher

rdflib.term.URIRef('http://linkedscience.org/teach/ns#Teacher')

Check out the specification to see which other terms are used within the TEACH namespace. http://linkedscience.org/teach/ns/#sec-specification. 
You can use a NamespaceManager to bind a prefix to a namespace: 

In [46]:
g.namespace_manager.bind('TEACH', URIRef('http://linkedscience.org/teach/ns#'))
TEACH.Teacher.n3(g.namespace_manager)

'TEACH:Teacher'

In [47]:
KRW = Namespace("http://krw.vu.nl/data#")

#creating individuals within your namespace
KRW.Teacher
KRW.Student

rdflib.term.URIRef('http://krw.vu.nl/data#Student')

**Exercise 4:**
1. create your own namespace (can be made up) 

In [48]:
AnimalSpace = Namespace("http://animal.com/")



## Creating RDF triples

Triples are added to the graph with the function Graph.add()

The parameter is a triple given in a Python **tuple** (subject, predicate, object)

Notice the namespace convenience syntax!

**Exercise 5:** 

1. create a new graph and add triples (~10) within your made-up namespace using Graph.add(). These triples can be about anything, for instance ingredients or recipes. Make sure they include the predicates RDF.type, RDFS.label and RDFS.subClassOf

2. open yourRDF.ttl, and write your triples out by hand in a syntax of your choice (turtle is recommended, notice the file extension!). Load the triples here with rdflib. 

In [49]:
animalGraph = Graph()
zebra = AnimalSpace.Zebra
penguin = AnimalSpace.Penguin
pig = AnimalSpace.Pig
chicken = AnimalSpace.Chicken

animalGraph.add((URIRef("Animal"), RDF.type, RDFS.Class))

animalGraph.add((pig, RDF.type, URIRef("Animal")))
animalGraph.add((penguin, RDF.type, URIRef("Animal")))
animalGraph.add((chicken, RDF.type, URIRef("Animal")))
animalGraph.add((zebra, RDF.type, URIRef("Animal")))

animalGraph.add((URIRef("Bipedidae"), RDFS.subClassOf, URIRef("Animal")))
animalGraph.add((URIRef("Quadruped"), RDFS.subClassOf, URIRef("Animal")))

animalGraph.add((penguin, RDFS.subClassOf, URIRef("Bipedidae")))
animalGraph.add((chicken, RDFS.subClassOf, URIRef("Bipedidae")))
animalGraph.add((pig, RDFS.subClassOf, URIRef("Quadruped")))
animalGraph.add((zebra, RDFS.subClassOf, URIRef("Quadruped")))

animalGraph.add((chicken, RDFS.label, Literal("Mary")))
animalGraph.add((pig, RDFS.label, Literal("Billy")))
animalGraph.add((penguin, RDFS.label, Literal("Ans")))
animalGraph.add((zebra, RDFS.label, Literal("Jozef")))

animalGraph.add((pig, FOAF.age, Literal("3")))
animalGraph.add((penguin, FOAF.age, Literal("10")))
animalGraph.add((chicken, FOAF.age, Literal("9")))
animalGraph.add((zebra, FOAF.age, Literal("15")))

animalGraph.serialize("./data/animalRDF.ttl")


<Graph identifier=Nbffe239a96c04f989cf1bad418a08082 (<class 'rdflib.graph.Graph'>)>

## Navigating graphs

rdflib uses iterators to navigate Graphs. The methods for navigating subjects, predicates and objects are Graph.subjects, Graph.predicates, Graph.objects

**Exercise 6:**

1. print all the triples in yourRDF.ttl
2. print all subjects in yourRDF.ttl
3. print all predicates in yourRDF.ttl
4. print all objects in yourRDF.ttl


In [50]:
for sub, pred, obj in animalGraph:
    print("Subject", sub)
    print("Predicate", pred)
    print("Object", obj)

# List all quadrupeds
for trip in animalGraph.triples((None, RDFS.subClassOf, Literal("Quadruped"))):
    print(trip)

Subject http://animal.com/Pig
Predicate http://www.w3.org/2000/01/rdf-schema#subClassOf
Object Quadruped
Subject http://animal.com/Penguin
Predicate http://www.w3.org/2000/01/rdf-schema#label
Object Ans
Subject http://animal.com/Chicken
Predicate http://xmlns.com/foaf/0.1/age
Object 9
Subject http://animal.com/Pig
Predicate http://www.w3.org/2000/01/rdf-schema#label
Object Billy
Subject http://animal.com/Zebra
Predicate http://xmlns.com/foaf/0.1/age
Object 15
Subject http://animal.com/Penguin
Predicate http://www.w3.org/2000/01/rdf-schema#subClassOf
Object Bipedidae
Subject Quadruped
Predicate http://www.w3.org/2000/01/rdf-schema#subClassOf
Object Animal
Subject http://animal.com/Chicken
Predicate http://www.w3.org/1999/02/22-rdf-syntax-ns#type
Object Animal
Subject http://animal.com/Penguin
Predicate http://www.w3.org/1999/02/22-rdf-syntax-ns#type
Object Animal
Subject http://animal.com/Zebra
Predicate http://www.w3.org/1999/02/22-rdf-syntax-ns#type
Object Animal
Subject http://animal

We can also filter the subjects, predicates and objects we want to retrieve, and match their values like in a database "join" operation


**Exercise 7:**

1. print all subject types in yourRDF.ttl
2. print all subject labels yourRDF.ttl

In [51]:
for lbl in animalGraph.objects(predicate=RDFS.label):
    print(lbl)

for tp in animalGraph.objects(predicate=RDF.type):
    print(tp)

Mary
Billy
Ans
Jozef
http://www.w3.org/2000/01/rdf-schema#Class
Animal
Animal
Animal
Animal


### Basic triple matching (almost querying!)

We use method Graph.triples and a Python tuple that acts as a mask for specifying our criteria

**Exercise 8:**

1. check whether a triple is in your graph -> print true or false
2. print all triples related to a certain subject in your graph
3. print all triples related to a certain object in your graph

In [52]:
for s,p,o in animalGraph.triples((None, None, None)):
    if s == AnimalSpace.Zebra:
        print(f"{s} is a Zebra!")

    if o == Literal("Quadruped"):
        print(f"{s} is a {o}")


http://animal.com/Zebra is a Zebra!
http://animal.com/Zebra is a Zebra!
http://animal.com/Zebra is a Zebra!
http://animal.com/Zebra is a Zebra!


## Assignment part 1: your own webapplication. 

You are a chef in a restaurant, and you need to serve someone that is gluten intolerant. 

1. load the ./data/recipes.rdf and ./data/ingredients.rdf datasets in one graph
2. query your graph (as we did in previous exercises) to retrieve all recipes without gluten
3. query your graph for all recipes that you can make for your gluten intolerant guest. 
4. the guest asks you whether there are more options. Can you find the recipes for which an ingredient with gluten can be replaced, solely using pattern matching? (Hint: you need to write multiple of these pattern matching queries, and check the predicate __substitutesFor__) 
5. another guest is allergic to pecan nuts, which recipes could you serve them (including those for which pecan nuts can be replaced) 

In [130]:
restaurantGraph = Graph()
restaurantGraph.parse("./data/recipes.rdf")
restaurantGraph.parse("./data/ingredients.rdf")

restaurantGraph.serialize("./data/restaurant_graph.ttl")

all_glutenfree_ingredients = []

restaurantGraph.namespace_manager.bind("WTM", URIRef("http://purl.org/heals/food/"))
WTM = Namespace("http://purl.org/heals/food/")

gluten_ingredients = []

for s in restaurantGraph.subjects(WTM.hasGluten, Literal("true", datatype=XSD.boolean)):
    gluten_ingredients.append(s)

# print(gluten_ingredients)

gluten_recipes = []

for s, o in restaurantGraph.subject_objects(WTM.hasIngredient):
    # Recipe contains gluten
    if o in gluten_ingredients:
        gluten_recipes.append(s)


gluten_free_recipes = []

for s in restaurantGraph.subjects(RDF.type, WTM.Recipe):
    if s not in gluten_recipes:
        gluten_free_recipes.append(s)


# Look for replacements
gluten_free_replacements = dict()
for gluten_ingredient in gluten_ingredients:
    for s, p, o in restaurantGraph.triples((None, WTM.substitutesFor, gluten_ingredient)):
        if o in gluten_free_replacements:
            gluten_free_replacements[o].append(s)
        else:
            gluten_free_replacements[o] = [s]


print(gluten_free_replacements)
        

for gluten_recipe in gluten_recipes:
    for s, p, o in restaurantGraph.triples((gluten_recipe, WTM.hasIngredient, None)):
        break
        print(o)
            

        


{rdflib.term.URIRef('http://purl.org/heals/ingredient/AllPurposeFlour'): [rdflib.term.URIRef('http://purl.org/heals/ingredient/GlutenFreeFlour'), rdflib.term.URIRef('http://purl.org/heals/ingredient/KamutFlour'), rdflib.term.URIRef('http://purl.org/heals/ingredient/WholeWheatFlour')]}
